In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

c:\Users\andre\anaconda3\envs\elsr-test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Hebbian(nn.Module):
    def __init__(self, input_size, output_size, learning_rate=0.001):
        super(Hebbian, self).__init__()
        self.linear = nn.Linear(input_size, output_size, bias=False)
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.linear(x)

    def update_weights(self, x, y):
        delta_w = torch.mm(x.t(), y) * self.learning_rate
        delta_w /= torch.norm(delta_w) + 1e-8
        self.linear.weight.data.add_(delta_w.t())

In [3]:
class HebbianVgg19(nn.Module):
    def __init__(self, num_classes=10):
        super(HebbianVgg19, self).__init__()
        vgg19 = torchvision.models.vgg19(pretrained=True)
        self.features = nn.Sequential(*list(vgg19.children())[:-1])
        for param in self.features.parameters():
            param.requires_grad = False
         
        self.classifier = Hebbian(25088, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        y = self.classifier(x)
        return y

In [4]:
# CIFAR10
transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# MNIST
'''transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.Grayscale(num_output_channels=3),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))
     ])'''

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = HebbianVgg19().to(device)

criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


c:\Users\andre\anaconda3\envs\elsr-test\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\andre\anaconda3\envs\elsr-test\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
from tqdm import tqdm

epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(trainloader, unit="batch") as tepoch:
        for i, data in enumerate(tepoch):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            net.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            output_grad = torch.zeros_like(outputs)
            for idx, l in enumerate(labels):
                output_grad[idx, l] = 1
            output_grad -= nn.functional.softmax(outputs, dim=1)
            output_grad /= output_grad.size(0)

            net.classifier.update_weights(net.features(inputs).view(inputs.size(0), -1).detach(), output_grad.detach())

            running_loss += loss.item()
            tepoch.set_postfix(loss=running_loss / (i + 1), accuracy=correct / total * 100)

    net.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0

    with torch.no_grad(), tqdm(total=len(testloader)) as pbar:
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

            test_loss += loss.item()
            pbar.update(1)

    print(f"Epoch {epoch + 1}, Train Loss: {running_loss / (i + 1)}, Train Acc: {correct / total * 100}. Test Loss: {test_loss / len(testloader)}, Test Acc: {test_correct / test_total * 100}")

print("Finished Training")

  1%|          | 100/10000 [00:35<58:56,  2.80it/s] 


Epoch 1, Train Loss: 2.616222856402397, Train Acc: 70.39. Test Loss: 2.107984893321991, Test Acc: 78.21000000000001


  1%|          | 100/10000 [00:33<55:51,  2.95it/s]


Epoch 2, Train Loss: 1.6891072260737419, Train Acc: 81.138. Test Loss: 2.329500181674957, Test Acc: 76.79


  1%|          | 100/10000 [00:34<56:43,  2.91it/s]


...

Epoch 8, Train Loss: 0.6355333993807435, Train Acc: 92.144. Test Loss: 2.1381836956739426, Test Acc: 82.78999999999999


  1%|          | 100/10000 [00:34<56:52,  2.90it/s]


Epoch 9, Train Loss: 0.5648845233730971, Train Acc: 92.824. Test Loss: 2.310579543709755, Test Acc: 82.36


  1%|          | 100/10000 [00:36<59:51,  2.76it/s] 

Epoch 10, Train Loss: 0.4835554581945762, Train Acc: 93.54400000000001. Test Loss: 2.883821804523468, Test Acc: 81.93
Finished Training
